In [12]:
# importing libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px

In [13]:
# making dfs of all files
lpi_df = pd.read_csv("../data/global-living-planet-index.csv")
rli_df = pd.read_csv("../data/red-list-index.csv")
ptbs_df = pd.read_csv("../data/protected-terrestrial-biodiversity-sites.csv")
rli_df.rename(columns={'15.5.1 - Red List Index - ER_RSK_LST': 'Red List Index'}, inplace=True)
ptbs_df.rename(columns={'15.1.2 - Average proportion of Terrestrial Key Biodiversity Areas (KBAs) covered by protected areas (%) - ER_PTD_TERR' : 'KBAs'}, inplace =True)

In [14]:
# making dictionary out of ipbes regions csv file 
dict_df = pd.read_csv("../data/ipbes_regions_subregions_1.1.csv")

# using dictionary comprehension by zipping list of country codes with list of country regions
# so region dct connects country code to it's ipbes region
region_dct = {code : region for (code, region) in zip(dict_df["GID_0"].tolist(), dict_df["Region"].tolist())}

In [15]:
""" # I first tried to use country name in the dictionary and area, but country code is standardized
rli_df["Region"] = rli_df["Code"].map(region_dct)
ptbs_df["Region"] = ptbs_df["Code"].map(region_dct)

# checking the null values after mapping to see if any countries were missed
print(ptbs_df[ptbs_df["Region"].isna()]["Entity"].unique(), 
rli_df[rli_df["Region"].isna()]["Entity"].unique())"""


# this (above) was my original code, but it was mapping incorrect values ot the null code values so I had to initialize the regions 
# as null so that it wouldn't map weirdly
rli_df["Region"] = np.nan
ptbs_df["Region"] = np.nan

# apply mapping only to non-null code values
rli_df.loc[rli_df["Code"].notna(), "Region"] = rli_df["Code"].map(region_dct)
ptbs_df.loc[ptbs_df["Code"].notna(), "Region"] = ptbs_df["Code"].map(region_dct)

# adding a region column to living planet index df so they're the same, and adding it in as "americas" 
# instead of separately to be the same as the other two
lpi_df["Region"] = lpi_df["Entity"].apply(lambda x: "Americas" if x in ["North America", "Latin America and the Caribbean"] else x)

In [ ]:
# looking for only overlapping years
print(lpi_df['Year'].unique())
print(ptbs_df['Year'].unique())
print(rli_df['Year'].unique())

# function that will filter each df to only apply the years that each of them have data for
# not applying this just yet because I don't want to edit the data until I'm using it
def overlapping_years(df):
    df = df[df['Year'].between(2000,2020)]
    return df

## COMPARING LIVING PLANET INDEX WITH REGIONS TO RED LIST INDEX

In [17]:
# making custom palettes to ensure that region line colors are standardized across both plots
lpi_palette = {
    "Africa": "cornflowerblue",
    "Americas": "green",
    "Europe and Central Asia": "red",
    "Asia and Pacific": "orange"
}
rli_palette = {
    "Africa": "cornflowerblue",
    "Americas": "green",
    "Europe and Central Asia": "red",
    "Asia and the Pacific": "orange"
}


In [ ]:
# filtering living planet index and red list index to only show the years that both have data on
filtered_lpi_df = lpi_df[lpi_df["Year"] >= 1993]
filtered_rli_df = rli_df[rli_df["Year"] <= 2020]

# initializing figure so I can make the two plots next to each other
fig, axes = plt.subplots(1, 2, figsize=(14, 6)) 

sns.lineplot(x='Year', y='Living Planet Index', 
             data=filtered_lpi_df[filtered_lpi_df["Region"].isin(["Africa", "Americas", "Europe and Central Asia", "Asia and Pacific"])], 
             hue='Region', errorbar=None, palette=lpi_palette, ax=axes[0])
axes[0].set_title("Living Planet Index")

sns.lineplot(x='Year', y="Red List Index", hue="Region", 
             data = filtered_rli_df[filtered_rli_df["Region"].notna()], palette=rli_palette, ax=axes[1])
axes[1].set_ylabel("Red List Index")
plt.suptitle("Living Planet and Red List Indices by IPBES Region from 1993 to 2020")
plt.savefig('livingplanetindex.png')
plt.show()

## GEO DATASET

##### IPBES regions and living planet index

In [19]:
ipbes = gpd.read_file("../data/IPBES_Regions_Subregions2.shp")

In [20]:
from shapely.validation import make_valid

# Had to get help from a friend for this one because the geodata wasn't dissolvign
ipbes["geometry"] = ipbes["geometry"].apply(lambda geom: make_valid(geom) if not geom.is_valid else geom)
ipbes["geometry"] = ipbes.simplify(tolerance=0.01) 
ipbes["geometry"] = ipbes["geometry"].buffer(0)
ipbes = ipbes[ipbes.is_valid]

In [21]:
ipbes_dissolved = ipbes.dissolve(by="Region")

In [22]:
def year_merge(year):
    # making df of just specified year 
    lpi_year = lpi_df[lpi_df["Year"] == year]
    
    # replacing name of asia and pacific so that merging later works
    lpi_year['Region'] = lpi_year['Region'].replace('Asia and Pacific', 'Asia and the Pacific')
    
    # getting mean of both north and south america since they're grouped together as "americas"
    americas = lpi_year[lpi_year['Region'] == 'Americas']
    mean = americas['Living Planet Index'].mean()
    # deleting old americas rows
    lpi_year = lpi_year[lpi_year['Region']!= 'Americas']
    
    # making new row a df so I can concatenate
    new_row = pd.DataFrame({'Living Planet Index' : [mean], 'Region' : ['Americas']})

    # concatenating both together
    lpi_year = pd.concat([lpi_year, new_row], ignore_index=True)
    # merging dataframes and making gdf
    year_gdf = gpd.GeoDataFrame(ipbes_dissolved.merge(lpi_year, left_on="Region", right_on="Region"))
    return year_gdf

In [ ]:
# This takes a second to run
gdf_2020 = year_merge(2020)

gdf_2020.plot(
    column = 'Living Planet Index',
    legend = True,
    figsize = (10,6),
    cmap = 'YlGn',
    legend_kwds = {'label' : 'Living Planet Index', 'orientation' : 'horizontal'}
)
plt.title('Living planet index by Region of the World in 2020')
plt.axis('off')
plt.savefig('lpibyregion_2020.png')
plt.show()

In [24]:
import numpy as np
pd.set_option('mode.chained_assignment', None)
avg_changes = {}

# loop through regions in data (making sure these are only regions that are present after use of year_merge function)
for region in year_merge(1970)['Region'].unique():
    percent_changes = []

    # loop through all years in dataset (except stopping at 2019, because we're looking ahead at percent change)
    for year in range(1970, 2020):
        # get data for current year and and filter region
        first = year_merge(year)[year_merge(year)["Region"] == region]
        second = year_merge(year + 1)[year_merge(year + 1)["Region"] == region]
        
        # calculate percent change of living planet index from current year to next and add to list of changes 
        change = (second["Living Planet Index"].iloc[0] - first["Living Planet Index"].iloc[0]) / first["Living Planet Index"].iloc[0]
        percent_changes.append(change)

    # add mean change to dictionary 
    avg_changes[region] = np.mean(percent_changes)

# making a dataframe so I can merge it with the geodataset and make a map
changes_df = pd.DataFrame(list(avg_changes.items()), columns=['Region', 'Average Change'])
change_gdf = gpd.GeoDataFrame(changes_df.merge(ipbes_dissolved, left_on="Region", right_on="Region"))



In [ ]:
fig = change_gdf.plot(
    column = "Average Change",
    legend = True,
    figsize = (10, 6),
    legend_kwds = {'label': 'Average Percent Change', 'orientation': 'horizontal'},
    cmap = 'OrRd'
)
# had to change background color so that eurasia was visible since it's so late 
plt.gcf().set_facecolor('lightsteelblue')
plt.axis('off')
plt.title("Average Yearly Percent Change in Living Planet Index from 1970 to 2020")
plt.savefig('lpi_percent_change.png')
plt.show()

##### Mean red list index 

In [28]:
countries = gpd.read_file("../data/world-administrative-boundaries.shp")
countries.loc[countries["color_code"].notna(), "ipbes_region"] = rli_df["Code"].map(region_dct)


In [ ]:
aggregated_rli_df = rli_df.groupby('Code')['Red List Index'].mean().reset_index()
gdf = gpd.GeoDataFrame(aggregated_rli_df.merge(countries, left_on='Code', right_on= 'color_code'))
gdf.plot(
    column='Red List Index', 
    legend=True, 
    figsize = (10, 6),
    legend_kwds = {'label':'Average Red List Index', 'orientation' : 'horizontal'},
    cmap='gist_heat',
    )
plt.axis('off')
plt.title("Average Red List Index from 1993 to 2020 by country")
plt.savefig('redlistmap.png')
plt.show()

#### Average proportion of Terrestrial Key Biodiversity Areas (KBAs) covered by protected areas (%)
## Map of change in proportion from 2023 to 2000 by percent points

In [30]:
# df of just 2000
ptbs_2000 = ptbs_df[ptbs_df["Year"] == 2000]
# add column to save 2000 data
ptbs_2000["KBAs 2000"] = ptbs_2000['KBAs']

# df of just 2023
ptbs_2023 = ptbs_df[ptbs_df["Year"] == 2023]
# add column to save 2023 data
ptbs_2023["KBAs 2023"] = ptbs_2023['KBAs']

# merge data with 2023 and 2000 column
change_df = ptbs_2000.merge(ptbs_2023, left_on='Code', right_on='Code')
ptbs_gdf = gpd.GeoDataFrame(change_df.merge(ipbes, left_on='Code', right_on='ISO_3'))

# add column to calculate and save change for each country
ptbs_gdf["Change"] = ptbs_gdf['KBAs 2023'] - ptbs_gdf["KBAs 2000"]

In [ ]:
ptbs_gdf.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm

# Example data
data = np.random.randn(10, 10)

# Load a diverging colormap (e.g., 'coolwarm')
cmap = plt.get_cmap('coolwarm')

# Define the midpoint where the colormap should diverge
midpoint = 0.0  # The value at which the colormap diverges (can adjust as needed)

# Create a TwoSlopeNorm with the desired midpoint
norm = TwoSlopeNorm(vmin=data.min(), vcenter=midpoint, vmax=data.max())

# Plot the data with the custom colormap and normalization
plt.imshow(data, cmap=cmap, norm=norm)
plt.colorbar()
plt.title('Diverging Colormap with Custom Midpoint')
plt.show()



In [ ]:
from matplotlib.colors import TwoSlopeNorm
midpoint = 0.0  # For example, 0 could be the neutral point where the colormap diverges

# Create a diverging colormap and apply TwoSlopeNorm with the custom midpoint
cmap = plt.get_cmap('coolwarm')  # Choose your colormap here
norm = TwoSlopeNorm(vmin=ptbs_gdf['Change'].min(), vcenter=midpoint, vmax=ptbs_gdf['Change'].max())

ptbs_gdf.plot(
    column = 'Change',
    cmap = cmap,
    norm=norm,
    legend=True,
    legend_kwds={'label': 'Value of KBA Proportion in 2023', 'orientation' : 'horizontal', 'shrink' : 0.5}
)
plt.axis('off')
plt.title("Change")
plt.savefig('protected-terrestrial-biodiversity-sites.png')

#### Regression

In [34]:
# dataFrames for each region for each data set 
africa_df = filtered_lpi_df[filtered_lpi_df["Region"] == "Africa"]
americas_df = filtered_lpi_df[filtered_lpi_df["Region"] == "Americas"]
europe_central_asia_df = filtered_lpi_df[filtered_lpi_df["Region"] == "Europe and Central Asia"]
asia_pacific_df = filtered_lpi_df[filtered_lpi_df["Region"] == "Asia and Pacific"]
africa_df_rli = filtered_rli_df[filtered_rli_df["Region"]== "Africa"]
americas_df_rli = filtered_rli_df[filtered_rli_df["Region"]== "Americas"]
european_central_asia_df_rli = filtered_rli_df[filtered_rli_df["Region"]== "Europe and Central Asia"]
asia_pacific_df_rli = filtered_rli_df[filtered_rli_df["Region"]== "Asia and the Pacific"]
# Define the regions you want to analyze
regions = ["Africa", "Americas", "Europe and Central Asia", "Asia and the Pacific"]

In [35]:
def plot_future_trend(region_df, region_name, model, ax, future_year_range, plot_actual=True):
    """
    Train and predict future Living Planet Index values for a region,
    and optionally plot both actual and predicted values.region_df: DataFrame containing data for the specific region.
    region_name: Name of the region (for labeling).,model: A scikit-learn regression model.,ax: Matplotlib Axes object to plot on.
    - future_year_range: List or array of years to predict future values for., plot_actual: Boolean flag to control whether to plot actual values as well. Defaults to True.
    """
    # Prepare features (using 'Year' only to predict future trend)
    X = region_df[['Year']]
    y = region_df['Living Planet Index']
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Generate future years for prediction
    future_years = np.array(future_year_range).reshape(-1, 1)
    
    # Predict values for the specified future years
    future_predictions = model.predict(future_years)
    
    # Print predicted values for the future years
    print(f"Predictions for {region_name}:")
    for year, prediction in zip(future_year_range, future_predictions):
        print(f"Year: {year}, Predicted Living Planet Index: {prediction:.3f}")
    
    # Optionally plot actual values
    if plot_actual:
        sns.lineplot(x='Year', y='Living Planet Index', data=region_df, ax=ax, label=f'{region_name} - Actual', marker='o')
    
    # Plot predicted values
    ax.plot(future_years.flatten(), future_predictions, label=f'{region_name} - Predicted', linestyle='--', marker='x')
    
    # Set plot titles and labels
    ax.set_title("Living Planet Index (Predicted by Region)")  # Removed 'World' reference
    ax.set_xlabel("Year")
    ax.set_ylabel("Living Planet Index")
    ax.legend()

In [ ]:
# Example usage
future_years = [2021, 2022, 2023, 2024, 2025]
regions = filtered_lpi_df['Region'].unique()  # Get unique regions from your data

# To plot both actual and predicted values (excluding 'World' and 'Freshwater'):
plot_all_regions(filtered_lpi_df, future_years, regions, plot_actual=True)

# To plot only predicted values (excluding 'World' and 'Freshwater'):
#plot_all_regions(filtered_lpi_df, future_years, regions, plot_actual=False)

In [ ]:
plot_all_regions(filtered_lpi_df, future_years, regions, plot_actual=False)

In [ ]:
def predict_future_trend(region_df, region_name, model, ax, all_predictions):
   """
    Predicts future trends for a specified region using a regression model and plots the results/
    Returns:
        tuple: A tuple containing:
            - future_years (pd.DataFrame): A DataFrame with years 2021 to 2025 for predictions.
            - future_predictions (np.ndarray): The predicted values for the next 5 years.""" 
    # Prepare features (using 'Year' only for fitting the model)
    X = region_df[['Year']]
    y = region_df["Red List Index"]
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Generate future years for prediction (2021-2025)
    future_years = pd.DataFrame({'Year': [2021, 2022, 2023, 2024, 2025]})
    
    # Predict values for the next 5 years
    future_predictions = model.predict(future_years)
    
    # Store predictions for normalization later
    all_predictions.extend(future_predictions)
    
    # Return data for plotting and annotation
    return future_years, future_predictions

## Red List Index Regressions

In [ ]:
regions = ["Africa", "Americas", "Europe and Central Asia", "Asia and the Pacific"]

# Filter the RLI data to only include data for the specified regions
filtered_rli_df = rli_df[rli_df["Region"].isin(regions)]

# Initialize the plot
sns.set(style="whitegrid")  
fig, ax = plt.subplots(figsize=(12, 8))

# Loop through each of the regions to plot actual and predicted data
for region in regions:
    # Filter data for the region up to 2024
    region_data = filtered_rli_df[(filtered_rli_df["Region"] == region) & (filtered_rli_df["Year"] <= 2024)]
    
    # Prepare the independent variable  and dependent variable
    X_train = region_data[['Year']]  
    y_train = region_data['15.5.1 - Red List Index - ER_RSK_LST'] 

    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict for the years 2025 to 2029
    future_years = pd.DataFrame({'Year': [2025, 2026, 2027, 2028, 2029]})
    future_predictions = model.predict(future_years)

    # Plot actual RLI data
    sns.lineplot(
        x=region_data['Year'],
        y=region_data['15.5.1 - Red List Index - ER_RSK_LST'],
        label=f'{region} - Actual',
        ax=ax
    )

    # Plot predicted RLI data
    sns.lineplot(
        x=future_years['Year'],
        y=future_predictions,
        label=f'{region} - Predicted',
        linestyle='--',
        ax=ax
    )

# plot labels 
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Red List Index', fontsize=12)
ax.set_title('Red List Index Predictions by Region for 2025–2029', fontsize=14)
ax.legend(fontsize=10, title="Legend", title_fontsize=12)

# Display the plot
plt.show()

In [ ]:

sns.set(style="whitegrid")  # Set seaborn style
fig, ax = plt.subplots(figsize=(12, 8))

# Loop through each region
for region in regions:
    # Filter data for the region up to 2024
    region_data = filtered_rli_df[(filtered_rli_df["Region"] == region) & (filtered_rli_df["Year"] <= 2024)]

    # Prepare the independent variable (Year) and dependent variable (Red List Index)
    X_train = region_data[['Year']]  # Year as the independent variable
    y_train = region_data['15.5.1 - Red List Index - ER_RSK_LST']  # Red List Index as the dependent variable

    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict for the years 2025 to 2029
    future_years = pd.DataFrame({'Year': [2025, 2026, 2027, 2028, 2029]})
    future_predictions = model.predict(future_years)

    # Print predictions for each year
    print(f"\nPredictions for {region}:")
    for year, pred in zip(future_years['Year'], future_predictions):
        print(f"Year {year}: Predicted RLI = {pred:.4f}")

    # Plot predicted RLI data with dotted lines and markers
    sns.lineplot(
        x=future_years['Year'],
        y=future_predictions,
        label=f'{region} - Predicted',
        linestyle='--',  # Dotted line
        marker='o',      # Marker for each year
        ax=ax
    )

# plot labels 
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Red List Index', fontsize=12)
ax.set_title('Red List Index Predictions by Region for 2025–2029', fontsize=14)
ax.legend(fontsize=10, title="Legend", title_fontsize=12)

# Display the plot
plt.show()